# 1번

In [1]:
import numpy as np

In [2]:
# input
input=np.array([[2,0,2,1,3,2],
            [0,2,0,2,2,2],
            [1,0,1,3,1,1],
            [0,0,1,1,1,0],
            [0,1,3,4,1,0],
            [0,1,0,0,5,2]])

In [3]:
# zero padding
input_=np.zeros((8,8))
input_[1:7,1:7]=input
input_

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 2., 0., 2., 1., 3., 2., 0.],
       [0., 0., 2., 0., 2., 2., 2., 0.],
       [0., 1., 0., 1., 3., 1., 1., 0.],
       [0., 0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 1., 3., 4., 1., 0., 0.],
       [0., 0., 1., 0., 0., 5., 2., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [4]:
# filter
filter=np.array([[1,0,0],
            [0,1,0],
            [0,0,1]])

In [10]:
# Convolution
row= input.shape[0]-filter.shape[0]+2*1+1
col= input.shape[1]-filter.shape[1]+2*1+1

result2=[]

for rn in range(row):
    for cn in range(col):
        result1=input_[rn:rn+filter.shape[0],cn:cn+filter.shape[1]]*filter
        result2.append(np.sum(result1))
output=np.array(result2).reshape(row,col)
output

array([[ 4.,  0.,  4.,  3.,  5.,  2.],
       [ 0.,  5.,  3.,  5.,  4.,  5.],
       [ 1.,  1.,  4.,  4.,  3.,  3.],
       [ 1.,  4.,  5.,  3.,  4.,  1.],
       [ 1.,  1.,  3., 10.,  4.,  1.],
       [ 0.,  1.,  1.,  3.,  9.,  3.]])

In [16]:
# max pooling

def max_pooling(x, stride):
    mp =[]
    for i in range(0,len(x)-1,stride):
        for j in range(0,len(x)-1,stride):
            mp.append(np.max(x[i:i+stride, j:j+stride]))
    
    mp = np.array(mp)
    return mp.reshape(stride+1, stride+1)

output_=max_pooling(output,2)
output_

array([[ 5.,  5.,  5.],
       [ 4.,  5.,  4.],
       [ 1., 10.,  9.]])

# 2번

### 1) 기존 CNN 모델

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import random
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [21]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device=='cuda':
    torch.cuda.manual_seed_all(777)

In [29]:
# hyperparameters
learning_rate=0.001
training_epochs=15
batch_size=100

#MNIST dataset
mnist_train=dsets.MNIST(root='MNIST_data/',
                       train=True,
                       transform=transforms.ToTensor(),
                       download=True)
mnist_test=dsets.MNIST(root='MNIST_data/',
                      train=False,
                      transform=transforms.ToTensor(),
                      download=True)

# dataset loader
data_loader=DataLoader(dataset=mnist_train,
                      batch_size=batch_size,
                      shuffle=True,
                      drop_last=True)

# class로 CNN모델 설계
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.keep_prob = 0.5
        
        # layer1
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1=nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        
        # layer2
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2=nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        
        # layer3
        # ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3=nn.Sequential(
            nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2,padding=1))
        
        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1=nn.Linear(4*4*128,625,bias=True)
        nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4=nn.Sequential(
            self.fc1,
            nn.ReLU(),
            nn.Dropout(p=1-self.keep_prob))
        
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=self.layer3(out)
        out=out.view(out.size(0),-1) # Flatten them for FC
        out=self.layer4(out)
        out=self.fc2(out)
        return out
    
# 모델 선언
model=CNN().to(device)

# cost, optimizer
criterion=nn.CrossEntropyLoss().to(device) #softmax is internally computed
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

# Train
total_batch=len(data_loader)

print('Learning Started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost=0
    
    for X, Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)
        
        hypothesis=model(X)
        cost=criterion(hypothesis,Y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost+= cost/total_batch
        
    print('Epoch:','%04d'%(epoch+1), 'cost=','{:.9f}'.format(avg_cost))

print('Learning finished')

Learning Started. It takes sometime.
Epoch: 0001 cost= 0.192842379
Epoch: 0002 cost= 0.050659336
Epoch: 0003 cost= 0.036905844
Epoch: 0004 cost= 0.028666601
Epoch: 0005 cost= 0.022915171
Epoch: 0006 cost= 0.019199846
Epoch: 0007 cost= 0.015928745
Epoch: 0008 cost= 0.015398595
Epoch: 0009 cost= 0.012690732
Epoch: 0010 cost= 0.010419538
Epoch: 0011 cost= 0.010241176
Epoch: 0012 cost= 0.008266630
Epoch: 0013 cost= 0.009403898
Epoch: 0014 cost= 0.006834675
Epoch: 0015 cost= 0.007971950
Learning finished


In [30]:
# Test
with torch.no_grad():
    
    X_test=mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test=mnist_test.test_labels.to(device)
    
    prediction=model(X_test)
    correct_prediction=torch.argmax(prediction,1)==Y_test 
    # torch.argmax(a,axis): a의 axis에 해당하는 값들 중 가장 큰 값의 index 반환
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\ProgramData\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9677000045776367


### 2) 수정한 CNN 모델
- layer1의 kernel_size=4, layer3의 stride=2
- layer3의 output size=(2,2,128)

In [31]:
# hyperparameters
learning_rate=0.001
training_epochs=15
batch_size=100

#MNIST dataset
mnist_train=dsets.MNIST(root='MNIST_data/',
                       train=True,
                       transform=transforms.ToTensor(),
                       download=True)
mnist_test=dsets.MNIST(root='MNIST_data/',
                      train=False,
                      transform=transforms.ToTensor(),
                      download=True)

# dataset loader
data_loader=DataLoader(dataset=mnist_train,
                      batch_size=batch_size,
                      shuffle=True,
                      drop_last=True)

# class로 CNN모델 설계
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2,self).__init__()
        self.keep_prob = 0.5
        
        # layer1
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 27, 27, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1=nn.Sequential(
            nn.Conv2d(1,32,kernel_size=4,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        
        # layer2
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2=nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2))
        
        # layer3
        # ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 4, 4, 128)
        #    Pool      ->(?, 2, 2, 128)
        self.layer3=nn.Sequential(
            nn.Conv2d(64,128,kernel_size=3,stride=2,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2,padding=1))
        
        # L4 FC 2x2x128 inputs -> 625 outputs
        self.fc1=nn.Linear(2*2*128,625,bias=True)
        nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4=nn.Sequential(
            self.fc1,
            nn.ReLU(),
            nn.Dropout(p=1-self.keep_prob))
        
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = torch.nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self,x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=self.layer3(out)
        out=out.view(out.size(0),-1) # Flatten them for FC
        out=self.layer4(out)
        out=self.fc2(out)
        return out
    
# 모델 선언
model=CNN2().to(device)

# cost, optimizer
criterion=nn.CrossEntropyLoss().to(device) #softmax is internally computed
optimizer=optim.Adam(model.parameters(),lr=learning_rate)

# Train
total_batch=len(data_loader)

print('Learning Started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost=0
    
    for X, Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)
        
        hypothesis=model(X)
        cost=criterion(hypothesis,Y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost+= cost/total_batch
        
    print('Epoch:','%04d'%(epoch+1), 'cost=','{:.9f}'.format(avg_cost))

print('Learning finished')

Learning Started. It takes sometime.
Epoch: 0001 cost= 0.270708263
Epoch: 0002 cost= 0.068952031
Epoch: 0003 cost= 0.046793386
Epoch: 0004 cost= 0.037261110
Epoch: 0005 cost= 0.030546447
Epoch: 0006 cost= 0.023847545
Epoch: 0007 cost= 0.022468038
Epoch: 0008 cost= 0.018700982
Epoch: 0009 cost= 0.016164649
Epoch: 0010 cost= 0.013824138
Epoch: 0011 cost= 0.015041474
Epoch: 0012 cost= 0.010649967
Epoch: 0013 cost= 0.011590094
Epoch: 0014 cost= 0.008392265
Epoch: 0015 cost= 0.011113419
Learning finished


In [32]:
# Test
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9883000254631042


- cost는 기존 모델보다 증가하였으나, 정확도 0.96 -> 0.98 로 오히려 상승
- 기존 모델이 과적합이 된듯 하다.